<a href="https://colab.research.google.com/github/nguepigit2020/StemAway_Recommender/blob/main/discourseScrapper_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# implement

1. Non-Repeating Links [Done]

2. Save Form Stats [Done]

3. Proper scroll to the bottom [Done]

4. Filter common phrases [Done]

5. Regex (A-Z,(,),-) [Done]

6. Differentiate between english and other languages [Working On]

7. Look over Data (Are the words being retreived okay?)[Working On (Almost done)]

8. Make code efficient (Any way you can make it so it runs a bit faster? Ex. Figuring out where timeouts are needed and where not)

9. Make code cleaner (comments, proper PEP, convert to pyfile, FIX UR GODAMN VARIABLE NAMES)

10. Remove any functions which you dont need into "extraFunc.py"

11. Upload to github
    

In [6]:
class discourseWordScraper:
    
    def __init__ (self, basicLink, websitePage, specificClasses = [], specificTags = [], specificIds = [], specificCus = [], dicCusClass = []):
        
    
        self.basicLink = basicLink
        self.websitePage = websitePage
        
        self.specificClasses = specificClasses
        self.specificTags = specificTags
        self.specificIds = specificIds

        self.bSoup = None

        
    def setVar(self, htmlData, websitePage):
        self.bSoup = BeautifulSoup(htmlData.strip(), 'html.parser')
        self.websitePage = websitePage
        
    def getByClass(self):
        
        tempData = list()
        
        for classTxt in self.specificClasses:
            tempData += self.bSoup.find_all(class_=classTxt)
            
            
        return ([x.text.split() for x in tempData], self.specificClasses)
    
    def getById(self):
        
        tempData = list()
        
        for idTxt in self.specificIds:
            tempData += self.bSoup.find_all(id=idTxt)
        return ([x.text.split('[,.\s]') for x in tempData], self.specificId)
        
            
    def getByTag(self):
        
        tempData = list()
        
        for tagTxt in self.specificTags:
            tempData += self.bSoup.find_all(tagTxt)
            
        
        return([x.text.split('[,.\s]') for x in tempData], self.specificTags)
        

    
    def getAllRelText(self):
       
        for script in self.bSoup(["script", "style"]):
            script.extract()
        
        text = "ALLTXT "+ self.bSoup.get_text()
        print(text)
        return ([text.replace('\n', " ").split('[,.\s]')], "AllTxt")
        
         
        
    def validateInput(self, tempData, typeCases, statsDict, repeating = True):
        
        blackList = ["http","img","<p>"] #<- NEEDS INPUT TO RUN
        regex = re.compile("[^a-zA-Z-/_']")
        finalInput = list()
        
        
        for rawSentence in tempData:
                rawInput = ([regex.sub('', string.lower()) for string in rawSentence])
                
                finalInput.extend([string for string in rawInput if any(blackTag in string for blackTag in blackList) == False and string != ""])                

        
        print(finalInput)
        
        print(statsDict["date-scraped"])
        finalInput.insert(0, statsDict["date-scraped"])
        finalInput.insert(1, (self.basicLink + self.websitePage + "/"))
        finalInput.insert(2, statsDict["created"])
        finalInput.insert(3, statsDict["last reply"])
        
        try: finalInput.insert(4, statsDict["replies"]) 
        except: finalInput.insert(4, statsDict["reply"])
        
        try: finalInput.insert(5, statsDict["views"])
        except: finalInput.insert(5, statsDict["view"])
        
        try: finalInput.insert(6, statsDict["users"]) 
        except: finalInput.insert(6, statsDict["user"])
        
        try:finalInput.insert(7, statsDict["likes"])
        except: finalInput.insert(7, statsDict["like"])
        
        
        try: finalInput.insert(8, statsDict["links"])
        except: finalInput.insert(8, statsDict["link"])
        
        finalInput.insert(9, typeCases)
    

            
            
        
        return finalInput
        
            
    def saveCSV(self, finalInput):
        
        with open('data.csv', 'r', encoding="utf-8") as dfr, open('data.csv', 'a+', newline='', encoding="utf-8") as dfw:
            
            
            writer = csv.writer(dfw)
            reader = csv.reader(dfr)
           
            writer.writerow(finalInput) 
 

In [7]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:

import requests
from bs4 import BeautifulSoup
import csv
import re
import random
import time
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

class formNavigator():
    
    def __init__ (self, basicLink, websitePage="", blackList = {}):
        
        self.basicLink = basicLink
        self.websitePage = websitePage
        self.currentLink = basicLink + websitePage
        
        self.linksStored = list()
        self.blackList = {set}
        
        self.bSoup = None
        
        selService = Service('/Users/idrissnguepi/Downloads/chromedriver')
        
        option = webdriver.ChromeOptions()
        option.add_argument('--incognito')
#         chrome_options.add_argument("--headless")
        
        self.driver = webdriver.Chrome(service=selService, options=option)
        
    
    
    def scrollPage(self, userRequest = "down",numOfTimes="infinite"):
        
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        
        
        if numOfTimes == "infinite":
        
            while True:

                if userRequest == "up": new_height = self.driver.execute_script("window.scrollTo(0,0)")
                elif userRequest == "down": new_height = self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                time.sleep(2)

                if new_height == last_height:
                    break

                last_height = new_height
            
        else:
                
            for num in range(int(numOfTimes)):
                
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                print("hello")
                time.sleep(2)
                


                
    
    
    def getPageHtml(self):
        
        self.driver.get(f'{self.currentLink}')
        time.sleep(2)
        
        pageMeta = ""
        
        self.scrollPage(userRequest="up")
        
        if "/t/" in self.currentLink:
            pageMeta = self.getPageMeta(self.driver.page_source)
            self.scrollPage(userRequest="down")            
            
        elif "/c/" in self.currentLink: 
            self.scrollPage(numOfTimes = "5")
            
        else:
            self.scrollPage(userRequest="down")
                
        
        with open("htmlStored.txt", "r+", encoding='utf-8') as htmlPage:
            htmlPage.write(self.driver.page_source) 
            
        
        self.bSoup = BeautifulSoup(self.driver.page_source, 'html.parser')
        return self.bSoup, self.basicLink, self.websitePage, pageMeta
    
    def getPageMeta(self,html):
        
        print(self.currentLink)

        now = datetime.now()
        self.bSoup = BeautifulSoup(html, 'html.parser')
        
        try:
            infoUList = self.bSoup.find("div", class_="topic-map")
            
            dateList = [date.get("title") for date in infoUList.find_all("span", class_="relative-date") if date.get("title") != None][0:2]
            time.sleep(0.5)
        
            
            liNumbers = [num.text for num in infoUList.find_all("span", class_="number") if num.text != None]
            time.sleep(0.5)
            
            desText = [decs.text for decs in infoUList.find_all("h4") if decs.text != None]
            metaDataDict = {desText[string]: (dateList + liNumbers)[string] for string in range(len(desText))}
            
            if "likes" not in metaDataDict and "like" not in metaDataDict: metaDataDict["likes"] = "0"
            if "links" not in metaDataDict and "link" not in metaDataDict: metaDataDict["links"]= "0"
            
            
            
            metaDataDict["date-scraped"] = now.strftime("%d/%m/%Y %H:%M:%S")
            print(metaDataDict)
                
            return metaDataDict
    
        except AttributeError as e:
            
            metaDataDict = {'date-scraped': f'{now.strftime("%d/%m/%Y %H:%M:%S")}','created': f'{self.bSoup.find("span", class_="relative-date").get("title")}', 'last reply': '0', 'replies': '0', 'views': 'Unknown', 'users': '1', 'likes': 'Unknown', 'links':'Unknown'}
            print(metaDataDict)
            
            return metaDataDict
            
    
    def getLink(self):
        
        self.blackList.add("/c/international")

        
        for aTag in self.bSoup.find_all("a"):
            
            try:
                aTag = (aTag.get("href"))
                
                

                if aTag.startswith("/t/") or aTag.startswith("/c/") :
                    self.linksStored.append(aTag)
   
                
            except AttributeError as a:
                continue
        
        self.linksStored = list(dict.fromkeys(self.linksStored))
                
    
    def setLink(self, userRequest):
        
        

        self.websitePage = random.choice([item for item in self.linksStored if isinstance(item, str) and userRequest in item])
        self.currentLink = self.basicLink + self.websitePage
        self.driver.get(self.currentLink)   
        
        self.linksStored.remove(self.websitePage)
        self.blackList.add(self.websitePage)
        
        return self.currentLink, userRequest
        

In [ ]:
formNav = formNavigator(basicLink="https://forums.unrealengine.com")

startPageData = formNav.getPageHtml()
wordScraper = discourseWordScraper(basicLink=startPageData[1], websitePage=startPageData[2], specificClasses=["cooked"] )


numIter = 0

while numIter != 4:
    
    
    if numIter >= 2: 
        
        formNav.setLink("/t/")
        
        formNav.scrollPage()
        data = formNav.getPageHtml()
        wordScraper.setVar(htmlData=str(data[0]), websitePage = data[2] )
        html = wordScraper.getByClass()
        wordScraper.saveCSV(finalInput=wordScraper.validateInput(tempData=html[0], typeCases=html[1], statsDict=data[3], repeating=True))
        
    
    else: 
        formNav.getLink()
        formNav.setLink("/c/")
        formNav.getPageHtml()
        
    numIter+=1